<a href="https://colab.research.google.com/github/morettitommaso/sql-exercises/blob/main/edb05_Foodmart_P2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caricamento dati nel DBMS

La cella qua sotto serve per per scaricare il database e interagire con il DBMS.

Se il DBMS semtte di funzionare, rieseguite la  cella di codice sottostante.

In [61]:
# Per favore, non modificate il codice di questa cella
#urldb='https://drive.google.com/uc?export=download&id=1AdEOuHUAROodt60z4JMs2qptTLHiPHqL'
urldb='https://dl.dropboxusercontent.com/s/ax0358bwt4poscl/db04foodmart.db.pez?dl=0'
fn='db04foodmart.db.pez'
pwd='studymat3r1al!'

import sqlite3
import urllib
import urllib.request
import zipfile
import os

import pandas as pd
pd.set_option('display.max_colwidth', None) # -1
pd.set_option('display.max_rows', 100)
from IPython.display import display, HTML

def downloadDBFile(urlFile, localFileName, pwd=''):
  # Dowloading the .zip file into the colab remote computation facility

  print('Downloading db file')
  try:
    urllib.urlretrieve(urlFile, localFileName) # Works in Python2
    print('Python2 detected')
  except AttributeError:
    print('Python 3 detected')
    with urllib.request.urlopen(urlFile) as response: # Works in Python3
      data = response.read()
      fd=open(localFileName,'wb')
      fd.write(data)
    fd.close()
  print('Download Done')
  if localFileName.endswith('.zip'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      zip_ref.extractall(localFileName)
      print('File %s extracted' % localFileName)
  elif localFileName.endswith('.pez'):
    zipFileName = localFileName
    localFileName = zipFileName[:-4]
    print('Unzipping the password protected file content')
    with zipfile.ZipFile(zipFileName, 'r') as zip_ref:
      #zip_ref.extractall(localFileName, pwd=bytes(pwd,'utf-8')) # it messes up the paths with password encrypted files
      zip_ref.extract(localFileName, pwd=bytes(pwd,'utf-8')) # localFileName: the name of the file in the .zip archive
      print('File %s extracted' % localFileName)

  localFileName = os.path.abspath(localFileName)
  print('File saved in '+localFileName)
  return localFileName
  print('Unzipping the password protected file content')


dbFileFullPath = downloadDBFile(urldb, fn, pwd=pwd)

def rq(query, threshold=100): # eq is for Run Query
    conn = sqlite3.connect(dbFileFullPath)
    if query.replace(' ','').replace('\n','') == '':
      return
    try:
        res = pd.read_sql(query,conn)
        conn.close()
        nrecords=res.shape[0]
        if nrecords<=threshold:
          display(res)
          print("%d records returned" % (nrecords) )
          #HTML("<b> %d records returned </b>" % (nrecords) )
          #return res

        else:
          display(res.iloc[:threshold,:])
          print("Too many records in the result, returned only the first %d" % (threshold))
          #return res.iloc[:threshold,:]

    except (pd.io.sql.DatabaseError, Exception)  as dbe: # (sqlite3.DatabaseError , sqlite3.OperationalError)
        print('Error')
        print(type(dbe))
        print(dbe)

Python 3 detected
Download Done
Unzipping the password protected file content
File db04foodmart.db extracted
File saved in /content/db04foodmart.db


# Database Foodmart

Foodmart è una catena di supermercati (fittizia), che opera negli Stati Uniti, Canada e Messico, è composta da punti vendita la cui tipologia spazia dai piccoli negozi a supermercati di ampie dimensioni. Il database contiene i dati delle vendite, delle spese dei clienti, dei costi e delle promozioni ... e tante altre informazioni, riferiti ad un arco temporale di alcuni anni.

Nel corso delle esercitazioni, analizzeremo prevalentemente la tabella **sales_fact_1998** (e le tabelle collegate) la quale contiene i dati delle vendite effettuate dai negozi della catena nell’anno 1998.

Ogni record della tabella sales_fact_1998 riporta i dati di un prodotto acquistato da una persona.

La tabella **sales_fact_1998** è composta dai seguenti attributi:

* product_id: foreign key che identifica il prodotto venduto
* time_id: foreign key che identifica il giorno in cui è avvenuta la vendita
* customer_id: foreign key che identifica il cliente
* promotion_id: foreign key che identifica l’eventuale promozione applicata
* store_id: foreign key che identifica il negozio in cui è avvenuta la vendita
* store_sales: prezzo unitario di vendita applicato
* store_cost: stima del costo unitario che il supermercato ha sostenuto per acquistare il prodotto, trasportarlo, …
* unit_sales: quantità di prodotto venduta

Del database foodmart fanno parte anche le tabelle elencate qui di seguito (tra parentesi le traduzioni in italiano):
* product (prodotti)
* product_class (classi di prodotti)
* store (negozi)
* time_by_day (informazioni sui giorni dell’anno)
* promotion (promozioni)
* customer (clienti)
* region (regione)

queste ultime tabelle contengono informazioni extra sulle vendite effettuate, per esempio le tipologie di prodotti venduti, il negozio che ha effettuato la vendita, i dati del cliente che ha effettuato l’acquisto, le date e così via. Le informazioni delle varie tabelle sono collegate alla tabella sales_fact_1998 e tra di loro per mezzo di chiavi. es.: product_id, time_id, ...



## Database Schema

Fate riferimento alle indicazioni che vi darà il docente





# Esercizi

Nei prossimi esercizi, vi sarà richiesto di analizzare le vendite del 1998, contenute nella tabella sales_fact_1998.

Le query seguenti servono per effettuare una valutazione delle attività di gestione della catena di supermercati. L’obiettivo della valutazione è identificare degli interventi per aumentare sia il fatturato sia il reddito dei supermercati.

 Dopo aver realizzato le query descritte nei punti seguenti, create ulteriori query liberamente, in modo da completare a vostro piacere l’attività di valutazione.

**Le indicazioni sui risultati sono riportate [qua](https://docs.google.com/document/d/1McwSDTpHqY89AVXk6mfAwS5pPWkNrKL6R3bV297crwI/edit?usp=sharing)**

# Attività di marketing

Le query seguenti permettono di analizzare le attività di marketing svolte


## (EDB05) 1

Ci sono dei prodotti venduti sottocosto nel 1998? I prodotti sottocosto sono quelli per cui il valore dell’attributo store_cost è maggiore del valore di store_sales

In [2]:
rq("""
SELECT  *
FROM sales_fact_1998 AS sf
WHERE sf.store_cost > sf.store_sales
""")

,product_id,time_id,customer_id,promotion_id,store_id,store_sales,store_cost,unit_sales


0 records returned


## (EDB05) 2

Individuate le promozioni che hanno fatto guadagnare di più nelle vendite del 1998. Il guadagno associato ad una promozione è dato dal guadagno originato dalle vendite dei beni oggetto della promozione. Per avere informazioni sulle promozioni guardate l’attributo sales_fact_1998.promotion_id e la tabella promotion. Visualizzate il guadagno totale generato e il nome di ogni promozione.
Nel calcolo, non considerate i costi presenti nella tabella promotion.

Visualizzate le promozioni in ordine decrescente di guadagno totale  (escludete dalla visualizzazione la non promozione indicata con il nome “No Promotion”).

In [18]:
rq("""

SELECT p.promotion_name, SUM((f.store_sales - f.store_cost)) AS tot_guadagno

FROM sales_fact_1998 AS f

JOIN promotion AS p
  ON f.promotion_id = p.promotion_id

WHERE p.promotion_name != 'No Promotion'
GROUP BY p.promotion_name
ORDER BY tot_guadagno DESC
limit 5;

""")

,promotion_name,tot_guadagno
0,Price Winners,9222.8936
1,Two Day Sale,8985.0185
2,Weekend Markdown,7395.7639
3,Go For It,6988.4387
4,Save-It Sale,6404.7704


5 records returned


# Margine e redditività

Le query seguenti permettono di analizzare come l’azienda costruisce utile e margine. Il margine è un indicatore definito con qs formula: **margine = (ricavo tot.– costo tot.) / ricavo tot.**


## (EDB05) 3

Individuate i 5 prodotti che offrono il margine maggiore.

Suggerimento: ordinate il risultato di una query e del risultato considerate solo le prime 5 tuple.


In [26]:
rq("""
SELECT p.product_name,
      SUM( f.store_sales - f.store_cost ) / SUM(store_sales) AS margin

FROM sales_fact_1998 as f

JOIN product AS p
      ON f.product_id = p.product_id

GROUP BY p.product_name
ORDER BY margin DESC
limit 5;
""")

,product_name,margin
0,Walrus White Zinfandel Wine,0.617092
1,Pleasant Canned String Beans,0.616749
2,Faux Products Mint Mouthwash,0.616341
3,Dual City Lox,0.616310
4,High Quality Paper Towels,0.616298


5 records returned


## (EDB05) 4

Individuate le 5 categorie di prodotti che offrono il margine maggiore.

La categoria di un prodotto è identificata dall’attributo 'product_class.product_category'.

In [30]:
rq("""
SELECT pc.product_category,
      SUM( f.store_sales - f.store_cost ) / SUM(store_sales) AS margin

FROM sales_fact_1998 as f

JOIN product AS p
JOIN product_class AS pc
      ON (f.product_id = p.product_id
      AND p.product_class_id = pc.product_class_id)

GROUP BY pc.product_category
ORDER BY margin DESC
limit 5;
""")

,product_category,margin
0,Canned Sardines,0.604613
1,Carbonated Beverages,0.602696
2,Drinks,0.602302
3,Canned Tuna,0.602156
4,Kitchen Products,0.602009


5 records returned


## (EDB05) 5

Individuate i 5 settori merceologici del supermercato che offrono il margine maggiore.

Il settore merceologico è identificata dall’attributo ‘product_class.product_department’.


In [34]:
rq("""
SELECT pc.product_department,
      SUM( f.store_sales - f.store_cost ) / SUM(store_sales) AS margin

FROM sales_fact_1998 as f

JOIN product AS p
      ON f.product_id = p.product_id

JOIN product_class AS pc
      ON p.product_class_id = pc.product_class_id

GROUP BY pc.product_department
ORDER BY margin DESC
limit 5;
""")

,product_department,margin
0,Snacks,0.600777
1,Beverages,0.600777
2,Household,0.600636
3,Starchy Foods,0.600036
4,Baking Goods,0.599851


5 records returned


## (EDB05) 6

Come il punto precedente, tuttavia visualizzate i product_department che offrono un margine superiore al valore di 0,6 (60%).


In [36]:
rq("""
SELECT pc.product_department,
      SUM( f.store_sales - f.store_cost ) / SUM(store_sales) AS margin

FROM sales_fact_1998 as f

JOIN product AS p
      ON f.product_id = p.product_id

JOIN product_class AS pc
      ON p.product_class_id = pc.product_class_id

GROUP BY pc.product_department
HAVING margin > 0.6

ORDER BY margin DESC;
""")

,product_department,margin
0,Snacks,0.600777
1,Beverages,0.600777
2,Household,0.600636
3,Starchy Foods,0.600036


4 records returned


# Analisi del porfoglio vendite

Le query seguenti permettono di analizzare le attività di vendita


## (EDB05) 7

In questo punto vi sarà richiesto di scrivere delle query che permettano di confrontare, per ogni supermercato della catena, le vendite del settore carni con il settore prodotti in scatola.

Questo esercizio sarà suddiviso in sottopunti.


#### (EDB05) 7.a

Individuate per ogni negozio il ricavo conseguito vendendo i prodotti del settore merceologico "Prodotti in scatola" (valore "Canned Products" dell’attributo product_department della tabella product_class).

La query deve visualizzare l'id del supermercato e il ricavo conseguente alla vendita dei prodotti del settore "Prodotti in scatola".

Per ogni supermercato deve apparire una sola riga.



In [41]:
rq("""
SELECT f.store_id,
      SUM( f.store_sales*f.unit_sales ) AS Ricavo

FROM sales_fact_1998 as f

JOIN product AS p
      ON f.product_id = p.product_id

JOIN product_class AS pc
      ON p.product_class_id = pc.product_class_id

WHERE pc.product_department = "Canned Products"

GROUP BY f.store_id
ORDER BY f.store_id
limit 5;
""")

,store_id,Ricavo
0,1,1015.09
1,2,9.63
2,3,815.24
3,4,1214.77
4,5,18.23


5 records returned


#### (EDB05) 7.b

Come la query precedente, analizzando però il settore carni (valore "Meat" dell’attributo product_department della tabella product_class).


In [42]:
rq("""
SELECT f.store_id,
      SUM( f.store_sales*f.unit_sales ) AS Ricavo

FROM sales_fact_1998 as f

JOIN product AS p
      ON f.product_id = p.product_id

JOIN product_class AS pc
      ON p.product_class_id = pc.product_class_id

WHERE pc.product_department = "Meat"

GROUP BY f.store_id
ORDER BY f.store_id
limit 5;
""")

,store_id,Ricavo
0,1,1457.93
1,2,30.49
2,3,961.25
3,4,1038.92
4,5,68.49


5 records returned


#### (EDB05) 7.c

NB: per svolgere questo punto è necessario utilizzare le query annidate

Costruite un'unica query che per ogni supermercato mostri:
* il valore delle vendite del settore "Prodotti in scatola";
* il valore delle vendite del settore carni;
* il rapporto tra le vendite del settore carni e le vendite del settore "Prodotti in scatola".

Suggerimento: per poter scrivere la query richiesta è necessario utilizzare le dei due punti precedenti  come query annidate.

L’esempio che segue mostra l’uso delle query annidate (o sottoquery):

~~~~
select ...
from tab_a, tab_b, (
  select store_id, sum(store_value*unit_sales)
  from sales_fact_1998 group by store_id) as tab_virtuale_c
  where tab_virtuale_c.store_id = tab_a.store_id
...
) as tab_virtuale_c, tab_d, ...
~~~~

La query esterna contiene al suo interno una query (la parte tra parentesi) il cui risultato viene usato come se fosse una tabella vera e propria e a cui si fa riferimento con il nome *tab_virtuale_c* nella query esterna. In generale, in una query SQL dopo la parola chiave from possono essere riportate tabelle reali oppure il risultato di una query (che in questo caso viene chiamato query annidata).


In [51]:
rq("""
SELECT s.store_id,
      c.c_Ricavo AS vend_carne,
      s.s_Ricavo AS ven_scatol,
      (c.c_Ricavo / s.s_Ricavo) AS carne_scatol

FROM  (SELECT f.store_id,
      SUM( f.store_sales*f.unit_sales ) AS s_Ricavo
      FROM sales_fact_1998 as f
      JOIN product AS p
            ON f.product_id = p.product_id
      JOIN product_class AS pc
            ON p.product_class_id = pc.product_class_id
      WHERE pc.product_department = "Canned Products"
      GROUP BY f.store_id) AS s,

      (SELECT f.store_id,
      SUM( f.store_sales*f.unit_sales ) AS c_Ricavo
      FROM sales_fact_1998 as f
      JOIN product AS p
            ON f.product_id = p.product_id
      JOIN product_class AS pc
            ON p.product_class_id = pc.product_class_id
      WHERE pc.product_department = "Meat"
      GROUP BY f.store_id) AS c

WHERE (s.store_id = c.store_id)
ORDER BY s.store_id
limit 5;

""")

,store_id,vend_carne,ven_scatol,carne_scatol
0,1,1457.93,1015.09,1.436257
1,2,30.49,9.63,3.166147
2,3,961.25,815.24,1.179101
3,4,1038.92,1214.77,0.855240
4,5,68.49,18.23,3.756994


5 records returned


#(EDB05) 8

Determinate il ricavo totale suddiviso per "state" (attributo store.store_state) e mostrate gli stati in ordine decrescente di ricavo (ogni "state" deve apparire una sola volta).

Per ogni riga del risultato, visualizzate anche il campo store_country oltre che il campo store_state.


In [54]:
rq("""
SELECT s.store_state,
      s.store_country,
      SUM( f.store_sales*f.unit_sales ) AS Ricavo

FROM sales_fact_1998 as f

JOIN store AS s
      ON f.store_id = s.store_id

GROUP BY s.store_state
ORDER BY Ricavo DESC
limit 5;
""")

,store_state,store_country,Ricavo
0,WA,USA,882323.57
1,CA,USA,510171.19
2,Zacatecas,Mexico,503675.41
3,OR,USA,429264.74
4,BC,Canada,329277.34


5 records returned


#(EDB05) 9

Identificate i prodotti che hanno fatto guadagnare di più nel 1998 (visualizzate product_id, product_name, e totale del guadagno conseguito nel 1998, visualizzate i prodotti in ordine decrescente).


In [57]:
rq("""

SELECT f.product_id,
       p.product_name,
       SUM((f.store_sales - f.store_cost)*f.unit_sales) AS guadagno

FROM sales_fact_1998 AS f

JOIN product AS p
    ON f.product_id = p.product_id

GROUP BY f.product_id
ORDER BY guadagno DESC
limit 5;
""")

,product_id,product_name,guadagno
0,274,Great Pumpernickel Bread,3166.3214
1,309,Super Extra Chunky Peanut Butter,2988.5634
2,1180,Horatio Frosted Cookies,2971.0200
3,175,High Top New Potatos,2969.1200
4,1380,Hilltop Mint Mouthwash,2961.5582


5 records returned


#(EDB05) 10

Considerando le vendite della tabella sales_fact_1998, limitando la ricerca ai soli negozi di tipo supermarket (store.store_type='Supermarket'), visualizzate una lista di supermercati (ogni supermercato deve apparire al massimo una sola volta) ordinata in base al rapporto (guadagno totale del supermercato) / (qt totale di oggetti venduti dal supermercato).


In [60]:
rq("""

SELECT f.store_id,
      SUM( (f.store_sales-f.store_cost) * f.unit_sales) / (SUM(f.unit_sales)) AS idk

FROM sales_fact_1998 AS f

JOIN store AS s
  ON f.store_id = s.store_id

WHERE s.store_type='Supermarket'

GROUP BY f.store_id
ORDER BY idk DESC
limit 5;

""")

,store_id,idk
0,24,4.312640
1,11,4.268505
2,15,4.245500
3,1,4.243960
4,16,4.237612


5 records returned


# Analisi Varie

#(EDB05) 11

Visualizza l’utile generato dalle diverse tipologie di clientela. Le tipologie di clienti sono identificate dall’attributo customer.member_card.

Nella tabella risultato ci dovrà essere un solo valore per ogni tipologia di member_card.



In [64]:
rq("""

SELECT c.member_card,
      SUM((f.store_sales-f.store_cost)*f.unit_sales)AS utile

FROM sales_fact_1998 AS f

JOIN customer AS c
    on f.customer_id = c.customer_id

GROUP BY c.member_card;


""")

,member_card,utile
0,Bronze,1.198595e+06
1,Golden,2.796144e+05
2,Normal,4.889828e+05
3,Silver,1.789049e+05


4 records returned


#(EDB05) 12

In questo esercizio saranno analizzati i margini dei settori merceologici nei diversi supermercati.

L’esercizio è diviso in sottopunti.


#### (EDB05) 12.a

Visualizzate per ogni coppia <supermercato, settore merceologico> il margine corrispondente.

I supermercati sono identificati dall’attributo store.store_id mentre i settori merceologici sono identificati product_class.product_department.

Nei risultati per ogni supermercato visualizzate anche la tipologia del supermercato (attributo store.store_type)


In [81]:
rq("""

SELECT f.store_id,
    s.store_type,
    pc.product_department AS prod_dep,
    (SUM( f.store_sales - f.store_cost ) / SUM(f.store_sales)) AS margin

FROM sales_fact_1998 AS f

JOIN store AS s
  ON f.store_id = s.store_id

JOIN product AS p
  ON f.product_id = p.product_id

JOIN product_class AS pc
  ON p.product_class_id = pc.product_class_id

GROUP BY s.store_id, store_type, pc.product_department
ORDER BY f.store_id;

""")

,store_id,store_type,prod_dep,margin
0,1,Supermarket,Alcoholic Beverages,0.605404
1,1,Supermarket,Baked Goods,0.592731
2,1,Supermarket,Baking Goods,0.600553
3,1,Supermarket,Beverages,0.601945
4,1,Supermarket,Breakfast Foods,0.604729
5,1,Supermarket,Canned Foods,0.594967
6,1,Supermarket,Canned Products,0.601906
7,1,Supermarket,Carousel,0.601777
8,1,Supermarket,Checkout,0.591164
9,1,Supermarket,Dairy,0.602497


Too many records in the result, returned only the first 100


#### (EDB05) 12.b

Visualizzate per ogni coppia <tipologia supermercato, settore merceologico> il margine corrispondente. Questa query calcola il margine di un certo settore merceologico per tutti i supermercati di una stessa tipologia.

A differenza del sottopunto precedente, il margine non va calcolato sul singolo supermercato ma sull’insieme di tutti i supermercati dello stesso tipo.

 Tenete presente che il margine non è una grandezza additiva (per calcolare il margine di un insieme di supermercati non si può sommare i margini dei singoli supermercati, ma occorre ricalcolare il margine sull’insieme di tutti i dati delle vendite).


In [71]:
rq("""

SELECT s.store_type,
    pc.product_department AS prod_dep,
    (SUM( f.store_sales - f.store_cost ) / SUM(store_sales)) AS margin

FROM sales_fact_1998 AS f

JOIN store AS s
  ON f.store_id = s.store_id

JOIN product AS p
  ON f.product_id = p.product_id

JOIN product_class AS pc
  ON p.product_class_id = pc.product_class_id

GROUP BY s.store_type, pc.product_department;


""")

,store_type,prod_dep,margin
0,Deluxe Supermarket,Alcoholic Beverages,0.598918
1,Deluxe Supermarket,Baked Goods,0.598875
2,Deluxe Supermarket,Baking Goods,0.599909
3,Deluxe Supermarket,Beverages,0.601447
4,Deluxe Supermarket,Breakfast Foods,0.599561
5,Deluxe Supermarket,Canned Foods,0.599677
6,Deluxe Supermarket,Canned Products,0.596985
7,Deluxe Supermarket,Carousel,0.593993
8,Deluxe Supermarket,Checkout,0.598276
9,Deluxe Supermarket,Dairy,0.599537


Too many records in the result, returned only the first 100


#### (EDB05) 12.c

NB: per svolgere questo punto è necessario utilizzare le query annidate

Usando le due query precedenti come sottoquery, individuate quelle coppie di <supermercato, settore merceologico> per i quali il margine è inferiore al 96% del margine calcolato su <tipologia supermercato, settore merceologico> cioé il margine del corrispondente settore merceologico calcolato su tutti i supermercati della stessa tipologia.

Per costruire la query dovete mettere in join i risultati delle query del sottopunto a) e del sottopunto b) e visualizzare solamente i dati dei supermercati che hanno un margine inferiore al 96% del margine dei supermercati della stessa categoria (quest'ultima informazione è presente nei risultati della query del punto b).


In [83]:
rq("""

SELECT prima.store_id,
      prima.store_type,
      prima.prod_dep,
      prima.margin AS marg_sing,
      seconda.margin AS marg_glob

FROM (

     SELECT

      f.store_id,
      s.store_type,
      pc.product_department AS prod_dep,
      (SUM( f.store_sales - f.store_cost ) / SUM(f.store_sales)) AS margin

      FROM sales_fact_1998 AS f
      JOIN store AS s
        ON f.store_id = s.store_id
      JOIN product AS p
        ON f.product_id = p.product_id
      JOIN product_class AS pc
        ON p.product_class_id = pc.product_class_id

      GROUP BY f.store_id, s.store_id, pc.product_department

      ) AS prima

INNER JOIN

      (

      SELECT
        s.store_type,
        pc.product_department AS prod_dep,
        (SUM( f.store_sales - f.store_cost ) / SUM(f.store_sales)) AS margin

      FROM sales_fact_1998 AS f
      JOIN store AS s
        ON f.store_id = s.store_id
      JOIN product AS p
        ON f.product_id = p.product_id
      JOIN product_class AS pc
        ON p.product_class_id = pc.product_class_id

      GROUP BY s.store_type, pc.product_department
      ) AS seconda

    ON prima.store_type = seconda.store_type
    AND prima.prod_dep = seconda.prod_dep

WHERE marg_sing < (0.96*marg_glob);
""")

,store_id,store_type,prod_dep,marg_sing,marg_glob
0,2,Small Grocery,Canned Products,0.561177,0.597261
1,2,Small Grocery,Seafood,0.563873,0.616984
2,5,Small Grocery,Eggs,0.570476,0.597254
3,9,Mid-Size Grocery,Carousel,0.575562,0.601803
4,22,Small Grocery,Carousel,0.546520,0.600474
5,22,Small Grocery,Checkout,0.593187,0.619957


6 records returned


# (EDB05) 13

L’esercizio è diviso in sottopunti

#### (EDB05) 13.a

Visualizzate, per ogni product_id, le quantità vendute nel primo trimestre del 1998.

Le informazioni sul trimestre si trovano in time_by_day.quarter, il primo trimestre è ’Q1’, il secondo trimestre è ‘Q2’, ...).

Nel risultato potete limitarvi a visualizzare il product_id, non serve il product_name. Per ogni product_id deve essere visualizzata una sola riga nella tabella risultato.


In [92]:
rq("""

SELECT
  f.product_id,
  SUM(unit_sales) AS tot_Q_Q1

FROM sales_fact_1998 AS f

JOIN time_by_day as t
ON f.time_id = t.time_id

WHERE t.quarter = "Q1"

GROUP BY f.product_id
ORDER BY tot_Q_Q1 DESC
limit 5;

""")

,product_id,tot_Q_Q1
0,1339,157
1,444,149
2,463,146
3,266,142
4,880,142


5 records returned


#### (EDB05) 13.b

Visualizzate, per ogngni product_id, le quantità vendute in tutto il 1998.

Nel risultato potete limitarvi a visualizzare il product_id, non serve il product_name.

Per ogni product_id deve essere visualizzata una sola riga nella tabella risultato.


In [93]:
rq("""

SELECT
  f.product_id,
  SUM(unit_sales) AS tot_Q

FROM sales_fact_1998 AS f

GROUP BY f.product_id
ORDER BY tot_Q DESC
limit 5;

""")

,product_id,tot_Q
0,277,458
1,846,423
2,1297,423
3,274,422
4,319,422


5 records returned


#### (EDB05) 13.c

NB: per svolgere questo punto è necessario utilizzare le query annidate.

Usando le due query precedenti come sottoquery, visualizzate, per ogni product_id, la quantità di beni venduta nel 1. trimestre, la quantità di beni venduta in tutto l’anno, e il rapporto tra queste due quantità.

Visualizzate i risultati in ordine crescente rispetto a product_id. Nel risultato potete limitarvi a visualizzare il product_id, non serve il product_name.


In [102]:
rq("""

SELECT q1.product_id,
      q1.tot_Q_Q1,
      q98.tot_Q,
      q1.tot_Q_Q1 * 1.0 / q98.tot_Q  AS RAPPORTO

FROM (

    SELECT
    f.product_id,
    SUM(unit_sales) AS tot_Q_Q1
    FROM sales_fact_1998 AS f
    JOIN time_by_day as t
    ON f.time_id = t.time_id
    WHERE t.quarter = "Q1"
    GROUP BY f.product_id

    ) AS q1

JOIN (

    SELECT
    f.product_id,
    SUM(unit_sales) AS tot_Q
    FROM sales_fact_1998 AS f
    GROUP BY f.product_id

    ) AS q98

 ON q1.product_id = q98.product_id

ORDER BY q1.product_id
limit 5;

""")

,product_id,tot_Q_Q1,tot_Q,RAPPORTO
0,1,60,148,0.405405
1,2,80,275,0.290909
2,3,71,266,0.266917
3,4,93,278,0.334532
4,5,68,279,0.243728


5 records returned


# (EDB05) 14

L’esercizio è diviso in sottopunti

#### (EDB05) 14.a

Visualizzate, per ogni customer_id, il numero medio di quantità acquistate per transazione.

Una transazione corrisponde ad un singolo record della tabella sales_fact_1998.

Nel risultato potete limitarvi a visualizzare il customer_id, non servono i nomi e cognomi dei clienti. Per ogni customer_id deve essere visualizzata una sola riga nella tabella risultato.


In [105]:
rq("""

SELECT
  f.customer_id,
  AVG(f.unit_sales)

FROM sales_fact_1998 AS f

GROUP BY f.customer_id
limit 5;

""")

,customer_id,AVG(f.unit_sales)
0,3,3.000000
1,6,3.500000
2,8,2.904762
3,9,3.000000
4,10,3.411765


5 records returned


#### (EDB05) 14.b

NB: query annidate

Visualizzate, per ogni customer_id il numero di transazioni effettuate (cioè il n. di record di sales_fact_1998) in cui il cliente ha acquistato un numero di beni superiore alla quantità mediamente acquistata dal cliente nelle sue transazioni (in altre parole, ogni cliente avrà una media diversa).

Si suggerisce di calcolare in una sottoquery le quantità medie per transazione dei clienti.

Ordinate il risultato in ordine crescente rispetto al customer_id.


In [107]:
rq("""

SELECT
  f.customer_id,
  medie.media,
  COUNT(*)

FROM sales_fact_1998 AS f
JOIN(
  SELECT
  f.customer_id,
  AVG(f.unit_sales) AS media
  FROM sales_fact_1998 AS f
  GROUP BY f.customer_id
  ) AS medie
ON f.customer_id = medie.customer_id

WHERE f.unit_sales > medie.media
GROUP BY f.customer_id
limit 5;
""")

,customer_id,media,COUNT(*)
0,3,3.000000,2
1,6,3.500000,4
2,8,2.904762,30
3,9,3.000000,3
4,10,3.411765,8


5 records returned


#### (EDB05) 14.c

NB: query annidate

Arricchite la query del punto precedente visualizzando anche, per ogni cliente, il numero totale delle transazioni effettuate e il rapporto tra numero di transazioni in cui si è acquistata una quanità superiore alla media e numero di transazioni totali.

Per farlo, si suggerisce di aggiungere alla query precedente una sottoquery che calcola, per ogni customer_id, il numero totale di transazioni svolte.

Ordinate il risultato in ordine crescente rispetto al customer_id.

In [117]:
rq("""

SELECT
  f.customer_id,
  medie.media,
  totale.total,
  COUNT(*) AS maggiore,
  COUNT(*)*1.0/totale.total AS rapporto

FROM sales_fact_1998 AS f

JOIN(
  SELECT
  f.customer_id,
  AVG(f.unit_sales) AS media
  FROM sales_fact_1998 AS f
  GROUP BY f.customer_id
  ) AS medie
ON f.customer_id = medie.customer_id

JOIN(
  SELECT
  f.customer_id,
  COUNT(*) AS total
  FROM sales_fact_1998 AS f
  GROUP BY f.customer_id
  ) AS totale
ON medie.customer_id = totale.customer_id

WHERE f.unit_sales > medie.media
GROUP BY f.customer_id
limit 5;
""")

,customer_id,media,total,maggiore,rapporto
0,3,3.000000,13,2,0.153846
1,6,3.500000,6,4,0.666667
2,8,2.904762,42,30,0.714286
3,9,3.000000,13,3,0.230769
4,10,3.411765,17,8,0.470588


5 records returned
